# LCEL
- LangChain Expression Language (LCEL)은 LangChain 라이브러리에서 제공하는 선언적 방식의 인터페이스로, 
  
  복잡한 LLM (Large Language Model) 애플리케이션을 구축하고 실행하기 위한 도구

* LCEL은 LLM, 프롬프트, 검색기, 메모리 등 다양한 컴포넌트를 조합하여 강력하고 유연한 AI 시스템을 구축 가능

<br>

#### 주요 특징
- **선언적 구문**: 복잡한 로직을 간결하고 읽기 쉬운 방식으로 표현
- **모듈성**: 다양한 컴포넌트를 쉽게 조합하고 재사용
- **유연성**: 다양한 유형의 LLM 애플리케이션을 구축
- **확장성**: 사용자 정의 컴포넌트를 쉽게 통합
- **최적화**: 실행 시 자동으로 최적화를 수행

<br>

<hr>

<br>

## RunnablePassthrough
- `invoke()` 메서드를 통해 입력된 데이터를 그대로 반환
-  다음과 같은 시나리오에서 유용
   - 데이터를 변환하거나 수정할 필요가 없는 경우
   - 파이프라인의 특정 단계를 건너뛰어야 하는 경우
   - 디버깅 또는 테스트 목적으로 데이터 흐름을 모니터링해야 하는 경우

<br>

- **Runnable = 입력 $\rightarrow$ 출력 변환을 수행하는, 체이닝 가능한 추상 연산자**
  - 어떤 입력(Input)을 받아서, 어떤 출력(Output)을 반환할 수 있는 실행 가능한 단위
  - LCEL 전체는 Runnable 들을 조합해 만든 실행 그래프

| LCEL 객체             | Runnable로서의 역할             |
| ------------------- | -------------------------- |
| `PromptTemplate`      | 입력 dict → 포맷된 문자열 변환       |
| `ChatModel`           | 문자열 → LLM 응답 반환            |
| `Retriever`           | query → relevant documents |
| `OutputParser`        | LLM 결과 → Python 객체로 변환     |
| `RunnableLambda`      | 아무 Python 함수도 Runnable화    |
| `RunnableParallel`    | 병렬 구동 Runnable             |
| `RunnableMap`         | 여러 key별 runnable 실행        |
| `RunnablePassthrough` | 입력을 그대로 전달                 |



<br>

### 데이터 전달
- `RunnablePassthrough` 는 입력을 변경하지 않고 그대로 전달하거나 추가 키를 더하여 전달
  - `assign`과 함께 호출된 `RunnablePassthrough(RunnablePassthrough.assign(...))`는 입력을 받아 `assign` 함수에 전달된 추가 인자를 더함
- `RunnableParallel` 클래스를 사용하여 병렬로 실행 가능한 작업을 정의

<br>



In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

In [3]:
runnable = RunnableParallel(
    # 전달된 입력을 그대로 반환하는 Runnable
    passed=RunnablePassthrough(),
    # 입력의 "num" 값에 3을 곱한 결과를 반환하는 Runnable
    extra=RunnablePassthrough.assign(mult=lambda x: x["num"] * 3),
    # 입력의 "num" 값에 1을 더한 결과를 반환하는 Runnable
    modified=lambda x: x["num"] + 1,
)

In [4]:
# {"num": 1}을 입력으로 Runnable을 실행
runnable.invoke({"num": 1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

<br>

#### 검색기 예제

In [5]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [6]:
vectorstore = FAISS.from_texts(
    [
        "테디는 랭체인 주식회사에서 근무를 하였습니다.",
        "셜리는 테디와 같은 회사에서 근무하였습니다.",
        "테디의 직업은 개발자입니다.",
        "셜리의 직업은 디자이너입니다.",
    ],
    embedding=OpenAIEmbeddings(),
)

retriever = vectorstore.as_retriever()

In [7]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [8]:
model = ChatOpenAI(model_name="gpt-4o-mini")

In [9]:
def format_docs(docs):
    return "\n".join([doc.page_content for doc in docs])

In [10]:
retrieval_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [11]:
retrieval_chain.invoke("테디의 직업은 무엇입니까?")

'테디의 직업은 개발자입니다.'

In [12]:
retrieval_chain.invoke("셜리의 직업은 무엇입니까?")

'셜리의 직업은 디자이너입니다.'

<br>

<hr>

<br>

## Runnable 구조(그래프) 검토
- LCEL로 `runnable`을 생성한 후에는 이를 검사하여 어떤일이 일어나고 있는지 파악

<br>

- 텍스트 데이터로부터 FAISS 벡터 저장소를 생성

In [13]:
vectorstore = FAISS.from_texts(
    ["Teddy is an AI engineer who loves programming!"],
    embedding=OpenAIEmbeddings(),
)

retriever = vectorstore.as_retriever()

- 벡터 저장소를 기반으로 `retriever`를 생성

In [14]:
template = """Answer the question based only on the following context:
{context}  

Question: {question}"""

prompt = ChatPromptTemplate.from_template(
    template
)

In [15]:
model = ChatOpenAI(model="gpt-4o-mini")

In [16]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

<br>

### 그래프 구성 확인

<br>

#### `chain.get_graph()`
- 체인의 각 노드와 노드 간의 연결을 나타내는 그래프 객체를 반환
- 그래프의 노드는 체인의 각 단계를 나타내며, `edge`는 단계 간의 데이터 흐름을 나타냄

<br>

- 체인의 노드

In [17]:
chain.get_graph().nodes

{'3cf3be516b214f25bd651cc71d5fbf6d': Node(id='3cf3be516b214f25bd651cc71d5fbf6d', name='Parallel<context,question>Input', data=<class 'langchain_core.runnables.base.RunnableParallel<context,question>Input'>, metadata=None),
 'f7272a26361b4909a58253463a9c7ca3': Node(id='f7272a26361b4909a58253463a9c7ca3', name='Parallel<context,question>Output', data=<class 'langchain_core.utils.pydantic.RunnableParallel<context,question>Output'>, metadata=None),
 'f5ff3b2b857843e4ad149a5f0f206b54': Node(id='f5ff3b2b857843e4ad149a5f0f206b54', name='VectorStoreRetriever', data=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001A72FD8EF60>, search_kwargs={}), metadata=None),
 '581262bf8f4842a7ae8ce1b439e7dc93': Node(id='581262bf8f4842a7ae8ce1b439e7dc93', name='Passthrough', data=RunnablePassthrough(), metadata=None),
 '56724b3603f7423391f2b5f9086156c3': Node(id='56724b3603f7423391f2b5f9086156c3', name='ChatPromptTemplate', dat

<br>

- 체인의 엣지

In [18]:
chain.get_graph().edges

[Edge(source='8e9b77ddf6dc4a9580343689d0ab8987', target='4866eacc67a640f7967bfef934ba1e15', data=None, conditional=False),
 Edge(source='4866eacc67a640f7967bfef934ba1e15', target='79c9f173efbf461aae6e72ef15086646', data=None, conditional=False),
 Edge(source='8e9b77ddf6dc4a9580343689d0ab8987', target='47a8487c9f4f423b95ce8f2f7a803671', data=None, conditional=False),
 Edge(source='47a8487c9f4f423b95ce8f2f7a803671', target='79c9f173efbf461aae6e72ef15086646', data=None, conditional=False),
 Edge(source='79c9f173efbf461aae6e72ef15086646', target='41751dca96424882967db9209ef1de0f', data=None, conditional=False),
 Edge(source='41751dca96424882967db9209ef1de0f', target='b97534ec2f4a4bca9a361695b64471c1', data=None, conditional=False),
 Edge(source='527be10f45eb4a5d86b4d91f109c3ec7', target='8a31207773eb45978fb20ae1f5909932', data=None, conditional=False),
 Edge(source='b97534ec2f4a4bca9a361695b64471c1', target='527be10f45eb4a5d86b4d91f109c3ec7', data=None, conditional=False)]

<br>

### Runnable 그래프 구조 검토


In [19]:
chain.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

<br>

### 프롬프트 가져오기

In [20]:
chain.get_prompts()

[ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}  \n\nQuestion: {question}'), additional_kwargs={})])]

<br>

<hr>

<br>

## `RunnableLambda`

<br>

### `RunnableLambda`
- **사용자 정의 함수를 실행할 수 있는 기능**
  - 예) 데이터 전처리, 계산, 외부 API와의 상호 작용

<br>

### 사용자 정의 함수를 실행
- **사용자 정의함수가 받을 수 있는 인자는 1개 뿐**

    **만약 여러 인수를 받는 함수로 구현하고 싶다면, 단일 입력을 받아들이고 이를 여러 인수로 풀어내야 함**

In [21]:
from operator import itemgetter

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

<br>

- 텍스트의 길이를 반환하는 함수

In [22]:
def length_function(text):
    return len(text)

<br>

- 두 텍스트의 길이를 곱하는 함수

In [23]:
def _multiple_length_function(text1, text2):
    return len(text1) * len(text2)

<br>

- 2개 인자를 받는 함수로 연결하는 wrapper 함수

In [24]:
def multiple_length_function(_dict):  
    return _multiple_length_function(_dict["text1"], _dict["text2"])

In [26]:
# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template("what is {a} + {b}?")
# ChatOpenAI 모델 초기화
model = ChatOpenAI()

chain1 = prompt | model

<br>

- 체인 구성

In [27]:
chain = (
    {
        "a": itemgetter("input_1") | RunnableLambda(length_function),
        "b": {"text1": itemgetter("input_1"), "text2": itemgetter("input_2")}
        | RunnableLambda(multiple_length_function),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [28]:
chain.invoke({"input_1": "bar", "input_2": "gah"})

'3 + 9 is equal to 12.'

<br>

### `RunnableConfig` 인자로 활용
- `RunnableLambda`는 선택적으로 `Runnableconfig`를 수용할 수 있으며
    
    **콜백, 태그 및 기타 구성 정보를 중첩된 실행에 전달할 수 있음**

<br>

In [29]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableConfig
import json

<br>

- `"{foo:: bar}"`라는 잘못된 JSON을 정상적인 구조로 수정

In [36]:
def parse_or_fix(text: str, config: RunnableConfig):
    # 텍스트를 수정하는 프롬프트 템플릿
    fixing_chain = (
        ChatPromptTemplate.from_template(
            "Fix the following text:\n\ntext\n{input}\n\nError: {error}"
            " Don't narrate, just respond with the fixed data."
        )
        | ChatOpenAI()
        | StrOutputParser()
    )
    
    for _ in range(3):
        try:
            return json.loads(text)
        
        except Exception as e:
            # 파싱 중 오류가 발생하면 수정 체인을 호출하여 텍스트를 수정
            text = fixing_chain.invoke({"input": text, "error": e}, config)
            print(f"config: {config}")
            
    return "Failed to parse"

In [34]:
from langchain_classic.callbacks import get_openai_callback

In [35]:
with get_openai_callback() as cb:
    output = RunnableLambda(parse_or_fix).invoke(
        input="{foo:: bar}",
        config={"tags": ["my-tag"], "callbacks": [cb]},
    )

    print(f"\n\n수정한결과:\n{output}")

config: {'tags': ['my-tag'], 'metadata': {}, 'callbacks': <langchain_core.callbacks.manager.CallbackManager object at 0x000001A77379BA70>, 'recursion_limit': 25, 'configurable': {}}


수정한결과:
{'foo': 'bar'}


<br>

<hr>

<br>

## LLM 체인 라우팅 (`RunnableLambda`, `RunnableBranch`)

<br>

### `RunnableBranch`
- 입력에 따라 동적으로 로직을 라우팅할 수 있는 도구
  
  $\rightarrow$ 입력 데이터의 특성에 기반하여 다양한 처리 경로를 유연하게 정의

- 복잡한 의사 결정 트리를 간단하고 직관적인 방식으로 구현할 수 있도록 도와주며, 이는 코드의 가독성과 유지보수성을 크게 향상
- 또한 런타임에 동적으로 분기 조건을 평가하고 적절한 처리 루틴을 선택할 수 있어, 시스템의 적응력과 확장성을 높여줌
- 입력 데이터의 다양성과 변동성이 큰 애플리케이션 개발에 매우 유용

<br>

### 입력에 따른 동적 로직 라우팅
- 라우팅을 통해 이전 단계의 출력이 다음 단계를 정의하는 비결정적 체인을 생성할 수 있음
- 라우팅은 LLM과의 상호 작용에 구조와 일관성을 제공하는 데 도움

<br>

### 라우팅을 수행하는 방법
1. `RunnableLambda` 에서 조건부로 실행한 객체를 반환 (권장)
2. `RunnableBranch`

<br>

### 예시
- 들어오는 질문이 '수학', '과학', 또는 '기타' 중 하나로 분류하는 Chain 생성


In [37]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

In [38]:
prompt = PromptTemplate.from_template(
    """주어진 사용자 질문을 `수학`, `과학`, 또는 `기타` 중 하나로 분류하세요. 한 단어 이상으로 응답하지 마세요.

<question>
{question}
</question>

Classification:"""
)

In [39]:
chain = (
    prompt
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

In [40]:
chain.invoke({"question": "2+2 는 무엇인가요?"})

'수학'

In [41]:
chain.invoke({"question": "작용 반작용의 법칙은 무엇인가요?"})

'과학'

In [42]:
chain.invoke({"question": "Google은 어떤 회사인가요?"})

'기타'

<br>

#### 3개의 하위 체인

In [43]:
math_chain = (
    PromptTemplate.from_template(
        """You are an expert in math. \
        Always answer questions starting with "깨봉선생님께서 말씀하시기를..". \
        Respond to the following question:

        Question: {question}
        Answer:
        """
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

In [44]:
science_chain = (
    PromptTemplate.from_template(
        """You are an expert in science. \
        Always answer questions starting with "아이작 뉴턴 선생님께서 말씀하시기를..". \
        Respond to the following question:

        Question: {question}
        Answer:
        """
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

In [45]:
general_chain = (
    PromptTemplate.from_template(
        """Respond to the following question concisely:

        Question: {question}
        Answer:
        """
    )
    | ChatOpenAI(model="gpt-4o-mini")
)

<br>

### 사용자 정의 함수 사용
- **LangChain에서 권장하는 방식이며, 서로 다른 출력 간의 라우팅을 위해 사용자 정의 함수를 `RunnableLambda` 로 래핑**



In [46]:
from operator import itemgetter
from langchain_core.runnables import RunnableLambda

In [47]:
def route(info):
    if "수학" in info["topic"].lower():
        return math_chain
    
    elif "과학" in info["topic"].lower():
        return science_chain
    
    else:
        return general_chain

In [48]:
full_chain = (
    {"topic": chain, "question": itemgetter("question")}
    | RunnableLambda(
        route
    )
    | StrOutputParser()
)

In [49]:
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

'깨봉선생님께서 말씀하시기를, 미적분은 수학의 한 분야로, 변화하는 양을 이해하고 분석하는 데 초점을 맞추고 있습니다. 미적분은 두 가지 주요 개념인 미분과 적분으로 나뉘어져 있습니다. \n\n미분은 함수의 변화율을 측정하는 과정으로, 그래프에서의 접선의 기울기를 구하는 방법이라고 할 수 있습니다. 즉, 어떤 함수가 주어졌을 때, 특정 점에서의 기울기를 파악하여 함수의 증가 또는 감소를 알 수 있게 해줍니다. 이는 물체의 속도나 가속도와 같은 변화하는 현상을 분석하는 데 매우 유용합니다.\n\n적분은 주어진 함수 아래의 면적을 계산하는 과정으로, 미분의 역과정이라고 볼 수 있습니다. 이는 일정 구간에서의 총량을 구하거나, 물리적인 의미에서 총합을 계산하는 데 사용됩니다. 예를 들어, 물체가 이동한 거리나 평균값을 구할 때 적분을 활용합니다.\n\n미적분은 이러한 미분과 적분의 원리를 통해 실제 문제를 해결하고, 자연현상을 수학적으로 모델링하는 기본 도구로 자리잡고 있습니다.'

In [50]:
full_chain.invoke({"question": "중력은 어떻게 작용하나요?"})

'아이작 뉴턴 선생님께서 말씀하시기를, 중력은 두 질량 간의 상호작용으로 발생하는 힘이며, 그 크기는 두 질량의 곱에 비례하고, 그들 사이의 거리의 제곱에 반비례한다는 것입니다. 이는 만유인력의 법칙으로 표현되며, 모든 물체는 서로에게 중력을 끌어당기는 힘을 발휘합니다. 이 힘은 지구와 같은 큰 천체가 물체를 끌어당기며, 우리가 지구 표면에 머물 수 있도록 하는 근본적인 원리입니다.'

In [51]:
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})

'RAG(Retrieval Augmented Generation)은 정보 검색(retrieval)과 생성(generation) 기법을 결합한 자연어 처리 모델입니다. 이 모델은 주어진 질문에 대한 답변을 생성하기 위해 외부 데이터베이스에서 관련 정보를 검색하고, 이 정보를 사용하여 더 정확하고 풍부한 응답을 생성합니다. RAG는 검색된 정보를 이용해 문맥을 제공함으로써, 보다 유용하고 정보가 풍부한 결과를 도출할 수 있도록 돕습니다.'

<br>

### `RunnableBranch`
- `RunnableBranch`는 입력값에 따라 실행할 조건과 `Runnable`을 정의할 수 있는 특별한 유형의 `Runnable`

<br>

#### 문법
- `RunnableBranch는` (조건, Runnable) 쌍의 리스트와 기본 `Runnable`로 초기화
- 호출 시 전달된 입력값을 각 조건에 전달하여 분기를 선택
- `True`로 평가되는 첫 번째 조건을 선택하고, 해당 조건에 해당하는 `Runnable`을 입력값과 함께 실행
- 제공된 조건과 일치하는 것이 없으면 기본 `Runnable` 을 실행

<br>

In [52]:
from operator import itemgetter
from langchain_core.runnables import RunnableBranch

In [54]:
branch = RunnableBranch(
    # 주제에 "수학"이 포함되어 있는지 확인하고, 포함되어 있다면 math_chain을 실행
    (lambda x: "수학" in x["topic"].lower(), math_chain),
    # 주제에 "과학"이 포함되어 있는지 확인하고, 포함되어 있다면 science_chain을 실행
    (lambda x: "과학" in x["topic"].lower(), science_chain),
    # 위의 조건에 해당하지 않는 경우 general_chain을 실행
    general_chain,
)

- 주제와 질문을 입력받아 branch를 실행하는 전체 체인을 정의


In [55]:
full_chain = (
    {"topic": chain, "question": itemgetter("question")} | branch | StrOutputParser()
)

In [56]:
full_chain.invoke({"question": "미적분의 개념에 대해 말씀해 주세요."})

'깨봉선생님께서 말씀하시기를 미적분은 수학의 두 중요한 영역인 미분과 적분을 포함하는 개념으로, 변화와 누적의 원리를 연구하는 학문입니다. 미분은 함수의 변화율을 측정하며, 특정 구간에서의 함수의 기울기, 즉 순간적인 변화량을 구하는 데 사용됩니다. 반면, 적분은 특정 구간에서 함수의 전체적인 값을 누적하고, 넓이나 부피를 계산하는 데 활용됩니다. 미적분의 기본정리는 이 두 가지 개념이 서로 밀접하게 연결되어 있음을 보여주며, 많은 과학 및 공학 분야에서 필수적인 도구로 사용됩니다.'

In [57]:
full_chain.invoke({"question": "중력 가속도는 어떻게 계산하나요?"})

'아이작 뉴턴 선생님께서 말씀하시기를, 중력 가속도는 물체가 지구의 중력에 의해 끌려내려가는 가속도를 나타내며, 일반적으로 약 9.81 m/s²로 알려져 있습니다. 중력 가속도를 계산하기 위해서는 다음과 같은 일반적인 방법을 사용할 수 있습니다.\n\n1. 물체의 질량(m)과 중력의 영향을 받는 높이(h)를 측정합니다.\n2. 중력 가속도(g)는 다음의 중력 법칙을 사용하여 계산할 수 있습니다: \n   \\[\n   g = \\frac{F}{m}\n   \\]\n   여기서 F는 중력의 힘으로, 이는 \\( F = m \\cdot g \\)라는 식을 다시 변형한 것입니다.\n3. 지구의 중력장은 평균적으로 약 9.81 m/s²로 일정하므로, 이 값을 직접 사용할 수도 있습니다.\n\n이러한 계산을 통해 중력 가속도를 확인할 수 있으며, 실험적 방법을 통해도 이를 측정할 수 있습니다.'

In [58]:
full_chain.invoke({"question": "RAG(Retrieval Augmented Generation)은 무엇인가요?"})

'RAG(Retrieval Augmented Generation)는 정보 검색과 자연어 생성 기술을 결합한 모델로, 주어진 질문에 대해 관련 정보를 외부 데이터베이스에서 검색하여 해당 정보를 토대로 응답을 생성하는 방식입니다. 이를 통해 보다 정확하고 풍부한 답변을 제공할 수 있습니다.'

<br>

<hr>

<br>

## `RunnableParallel`

<br>

### 입력 및 출력 조작
- `RunnableParallel` 은 시퀀스 내에서 하나의 `Runnable` 의 출력을 다음 `Runnable` 의 입력 형식에 맞게 조작하는 데 유용하게 사용

<br>

In [59]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [60]:
vectorstore = FAISS.from_texts(
    ["Teddy is an AI engineer who loves programming!"], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-4o-mini")

In [62]:
retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("What is Teddy's occupation?")

"Teddy's occupation is an AI engineer."

<br>

- `Runnable`과 함께 `RunnableParallel`을 구성할 때, 유형 변환이 자동으로 처리되므로, `RunnableParallel`클래스에서 입력으로 주입되는 `dict`입력을 별도 래핑할 필요가 없음

<br>

- 아래의 3가지 방식은 모두 동일하게 처리

```
- 자체 RunnableParallel 로 래핑됨
1. {"context": retriever, "question": RunnablePassthrough()}

2. RunnableParallel({"context": retriever, "question": RunnablePassthrough()})

3. RunnableParallel(context=retriever, question=RunnablePassthrough())
```

<br>

### `itemgetter`를 단축어로 사용
- `RunnableParallel`과 결합할 때 Python의 `itemgetter`를 단축어로 사용하여 `map`에서 데이터를 추출 가능

In [63]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [64]:
vectorstore = FAISS.from_texts(
    ["Teddy is an AI engineer who loves programming!"], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

In [65]:
chain.invoke({"question": "What is Teddy's occupation?", "language": "Korean"})

'테디의 직업은 AI 엔지니어입니다.'

<br>

### 병렬처리를 단계별로 이해
- `RunnableParallel`을 사용하면 여러 `Runnable`을 병렬로 실행하고, 이러한 `Runnable`의 출력을 맵(`map`)으로 반환하는 것이 용이해짐

In [66]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel
from langchain_openai import ChatOpenAI

In [67]:
model = ChatOpenAI() 

capital_chain = (
    ChatPromptTemplate.from_template("{country} 의 수도는 어디입니까?")
    | model
    | StrOutputParser()
)

area_chain = (
    ChatPromptTemplate.from_template("{country} 의 면적은 얼마입니까?")
    | model
    | StrOutputParser()
)

map_chain = RunnableParallel(capital=capital_chain, area=area_chain)

map_chain.invoke({"country": "대한민국"})

{'capital': '대한민국의 수도는 서울 입니다.', 'area': '대한민국의 면적은 약 100,363제곱킬로미터 입니다.'}

<br>

- `chain`별로 입력 템플릿 변수가 달라도 상관없이 실행 가능

In [68]:
capital_chain2 = (
    ChatPromptTemplate.from_template("{country1} 의 수도는 어디입니까?")
    | model
    | StrOutputParser()
)

area_chain2 = (
    ChatPromptTemplate.from_template("{country2} 의 면적은 얼마입니까?")
    | model
    | StrOutputParser()
)

map_chain2 = RunnableParallel(capital=capital_chain2, area=area_chain2)

map_chain2.invoke({"country1": "대한민국", "country2": "미국"})

{'capital': '대한민국의 수도는 서울입니다.', 'area': '미국의 면적은 약 9.8백만 제곱 킬로미터입니다.'}

<br>

### 병렬 처리
- `RunnableParallel` 은 맵에 있는 각 `Runnable` 이 병렬로 실행되기 때문에 독립적인 프로세스를 병렬로 실행하는 데에도 유용

In [69]:
%%timeit

area_chain.invoke({"country": "대한민국"})

649 ms ± 145 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%%timeit

capital_chain.invoke({"country": "대한민국"})

658 ms ± 369 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [71]:
%%timeit

map_chain.invoke({"country": "대한민국"})

836 ms ± 271 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<br>

<hr>